In [3]:
!pip install -r  requirements.txt

Keyring is skipped due to an exception: 'keyring.backends'


# Get Lineup Metadata

In [19]:
import sys
sys.path.append('code')

from fantasyData import *
from draftkings import *

with open('config/fantasydata.json', 'r') as f:
        fdata_creds = json.load(f)
        
fd = fantasyData(fdata_creds['access_key'])

slate = fd.get_slate(date = (dt.datetime.now() - dt.timedelta(days= 1)).strftime('%Y-%b-%d'))
main_slate = slate[slate.OperatorName == 'Main']
#main_slate.head(10)

### Get Main Slate Player List

main_lineup = pd.DataFrame(main_slate.DfsSlatePlayers.values[0])
#display(main_lineup.head())

main_lineup = main_lineup[[
    'OperatorPlayerName',
    'OperatorSlatePlayerID',
    'OperatorPosition',
    'OperatorSalary', 
    'Team', 
    'OperatorRosterSlots'
]]

main_lineup.columns = [
    'Name', 
    'ID', 
    'Position', 
    'Salary', 
    'TeamAbrev', 
    'Roster Position'
]

display(main_lineup.head())

from tqdm.notebook import tqdm
log_dict = {}
for player in tqdm(main_lineup.Name): 
        
        try: 
            log_dict[player] = {}
            player_log = get_player_info(player).set_index('Date')
            try: 
                player_log.index = pd.to_datetime([x[0] for x in player_log.index])
            except: 
                player_log.index = pd.to_datetime(player_log.index)
            player_log['G'] = player_log['G'].astype(float)
            player_log.dropna(subset= ['G'], inplace=True)
                      
            player_log = player_log.loc[~player_log.index.duplicated(keep = 'first'),~player_log.columns.duplicated()]
            player_log.drop('Unnamed: 0', axis = 1, inplace = True)
            player_log = player_log.rename({
                'Unnamed: 5': 'Home/Away',
                'Unnamed: 7': 'W/L(+/-)'
            }, axis = 1)
            
        except Exception as e: 
            # print(player, f'error processing data ')
            # print(e)
            # display(player_log)
            # print(player_log.index)
            pass
            #continue
        
        try: 
            first_game_dates = player_log[player_log['G'] == 1].index
            last_season_start, this_season_start  = first_game_dates[-2], first_game_dates[-1]

            last_season  = player_log.loc[last_season_start: this_season_start - dt.timedelta(days = 1), :]    
            cur_season  = player_log.loc[this_season_start:, :]

            log_dict[player]['last_season'] = last_season
            log_dict[player]['cur_season'] = cur_season
            
        except Exception as e: 
            print(player, f'\nerror log:\n {e}')
            
#             display(player_log)
#             display(first_game_dates)
#             print(log_dict[player])
            break
            
lineup_log = pd.DataFrame.from_dict(log_dict, orient = 'index').reset_index().rename({'index': 'Name'},axis=1)
lineup_metadata = pd.merge(main_lineup, lineup_log, on = 'Name')

https://api.sportsdata.io/api/nba/fantasy/json/DfsSlatesByDate/2022-Dec-14


,Name,ID,Position,Salary,TeamAbrev,Roster Position
0,Luka Doncic,25902369,PG,12600,DAL,"[PG, G, UTIL]"
1,Nikola Jokic,25902372,C,11800,DEN,"[C, UTIL]"
2,Stephen Curry,25902374,PG,11000,GS,"[PG, G, UTIL]"
3,Pascal Siakam,25902377,PF/C,10500,TOR,"[PF, C, F, UTIL]"
4,Shai Gilgeous-Alexander,25902387,PG,10000,OKC,"[PG, G, UTIL]"


In [23]:
last_season_avg = pd.DataFrame([x.mean() for x in lineup_metadata.last_season])
last_season_avg.columns = [f'{x}_prevSeason' for x in last_season_avg.columns]

In [24]:
last_season_avg.describe()

,Rk_prevSeason,G_prevSeason,GS_prevSeason,FG_prevSeason,FGA_prevSeason,3P_prevSeason,3PA_prevSeason,FT_prevSeason,FTA_prevSeason,ORB_prevSeason,DRB_prevSeason,TRB_prevSeason,AST_prevSeason,STL_prevSeason,BLK_prevSeason,TOV_prevSeason,PF_prevSeason,PTS_prevSeason,ORtg_prevSeason,DRtg_prevSeason
count,333.000000,333.000000,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02,3.330000e+02
mean,38.281636,34.348348,4.781393e+71,8.028445e+106,6.409882e+146,9.432379e+70,3.815504e+105,2.269330e+89,2.269378e+97,7.072999e+69,4.509233e+119,5.341272e+134,5.846820e+113,5.218045e+71,4.822495e+70,6.855853e+71,7.922942e+71,7.605952e+145,4.743894e+211,4.610019e+222
std,1.625998,1.667452,7.945634e+72,1.089371e+108,1.169694e+148,1.537857e+72,6.962633e+106,4.141139e+90,4.141226e+98,8.293858e+70,8.228578e+120,9.746906e+135,1.066944e+115,9.268089e+72,8.621249e+71,9.461574e+72,1.431905e+73,1.387528e+147,inf,inf
min,17.103448,15.000000,3.831418e+26,2.981709e+36,8.314318e+55,7.052493e+25,3.589967e+34,1.937600e+32,1.911802e+34,3.483103e+24,2.111499e+27,2.114982e+28,2.081058e+31,3.452070e+26,3.448660e+18,4.284907e+26,1.075977e+27,6.962490e+54,3.714155e+73,3.455407e+80
25%,38.308824,34.500000,1.633987e+65,9.939867e+89,2.526680e+132,1.504606e+64,6.506670e+65,4.413296e+65,4.417724e+69,1.502941e+64,1.013395e+78,1.028119e+89,1.275234e+96,6.194311e+65,2.941340e+62,5.003265e+65,5.033987e+65,2.228091e+130,1.409113e+189,1.323253e+184
50%,38.308824,34.500000,1.633987e+65,9.939867e+89,2.526680e+132,1.504606e+64,6.506670e+65,4.413296e+65,4.417724e+69,1.502941e+64,1.013395e+78,1.028119e+89,1.275234e+96,6.194311e+65,2.941340e+62,5.003265e+65,5.033987e+65,2.228091e+130,1.409113e+189,1.323253e+184
75%,38.308824,34.500000,1.633987e+65,9.939867e+89,2.526680e+132,1.504606e+64,6.506670e+65,4.413296e+65,4.417724e+69,1.502941e+64,1.013395e+78,1.028119e+89,1.275234e+96,6.194311e+65,2.941340e+62,5.003265e+65,5.033987e+65,2.228091e+130,1.409113e+189,1.323253e+184
max,46.970588,39.000000,1.443001e+74,1.777007e+109,2.134490e+149,2.792252e+73,1.270561e+108,7.556870e+91,7.557028e+99,1.302599e+72,1.501575e+122,1.778643e+137,1.946991e+116,1.690911e+74,1.573016e+73,1.585988e+74,2.613001e+74,2.532004e+148,1.474691e+214,1.533872e+225


In [74]:
lineup.last_season.values[0]#.dropna()

,Rk,G,Age,Tm,Home/Away,Opp,W/L(+/-),GS,MP,FG,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
2021-10-21,1,1.0,22-235,DAL,@,ATL,L (-26),1,35:24,6,...,32.4,14.2,41.5,1.3,0.0,21.0,28.3,87,109,2.0
2021-10-23,2,2.0,22-237,DAL,@,TOR,W (+8),1,38:53,11,...,23.1,11.2,60.7,0.0,2.6,16.0,35.7,104,102,6.5
2021-10-26,3,3.0,22-240,DAL,NaN,HOU,W (+10),1,34:16,10,...,35.8,19.8,33.8,2.8,3.0,12.8,37.0,101,96,3.4
2021-10-28,4,4.0,22-242,DAL,NaN,SAS,W (+5),1,34:20,11,...,8.9,4.5,27.3,0.0,2.4,25.6,35.8,95,107,0.1
2021-10-29,5,5.0,22-243,DAL,@,DEN,L (-31),1,26:25,5,...,31.1,12.5,52.2,3.7,0.0,9.0,38.5,79,102,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,78,61.0,23-032,DAL,@,WAS,L (-32),1,36:17,11,...,23.7,11.4,40.8,0.0,2.3,15.9,40.6,121,137,8.5
2022-04-03,79,62.0,23-034,DAL,@,MIL,W (+6),1,39:04,9,...,21.4,13.3,66.0,3.8,0.0,17.6,42.0,114,109,8.6
2022-04-06,80,63.0,23-037,DAL,@,DET,W (+18),1,33:57,8,...,22.5,13.6,58.8,1.5,0.0,18.3,44.1,107,118,-2.7
2022-04-08,81,64.0,23-039,DAL,NaN,POR,W (+50),1,29:36,12,...,38.0,22.3,50.4,0.0,3.2,16.2,46.9,135,81,18.2


In [59]:
lineup.last_season.values[0][[x for x in lineup.last_season.values[0].columns if 'Unnamed' in x]]

,Unnamed: 0,Unnamed: 5,Unnamed: 7
2021-10-21,0.0,@,L (-26)
2021-10-23,1.0,@,W (+8)
2021-10-26,2.0,NaN,W (+10)
2021-10-28,3.0,NaN,W (+5)
2021-10-29,4.0,@,L (-31)
...,...,...,...
2022-04-03,81.0,@,W (+6)
2022-04-06,82.0,@,W (+18)
2022-04-08,84.0,NaN,W (+50)
2022-04-10,85.0,NaN,W (+10)


In [51]:
player_log.columns

Index(['Unnamed: 0', 'Rk', 'G', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7',
       'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc',
       '+/-', 'Unnamed: 0', 'Rk', 'G', 'Age', 'Tm', 'Unnamed: 5', 'Opp',
       'Unnamed: 7', 'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'],
      dtype='object')

In [44]:
player_log

,game_#,Rk,G,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,BPM
2003-10-29,0.0,1,1.0,18-303,CLE,@,SAC,L (-14),1,42:50,...,8.6,39.1,5.0,0.0,8.6,25.6,123,112,24.7,10.8
2003-10-30,1.0,2,2.0,18-304,CLE,@,PHO,L (-9),1,40:21,...,14.9,33.9,1.2,0.0,25.8,28.5,89,97,14.7,1.0
2003-11-01,2.0,3,3.0,18-306,CLE,@,POR,L (-19),1,39:10,...,6.6,23.6,2.7,0.0,13.4,18.2,74,112,5.0,-6.5
2003-11-05,3.0,4,4.0,18-310,CLE,NaN,DEN,L (-4),1,41:06,...,13.1,27.8,2.5,5.1,14.9,13.7,85,95,11.2,2.0
2003-11-07,4.0,5,5.0,18-312,CLE,@,IND,L (-1),1,43:44,...,6.3,12.0,0.0,0.0,24.9,30.7,85,112,9.0,-7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,NaN,23,17,37-341,LAL,@,CLE,L (-14),1,35:37,...,11.7,23.7,1.3,0.0,15.3,37.8,97,109,17.0,-3.1
2022-12-07,NaN,24,NaN,37-342,LAL,@,TOR,L (-13),Inactive,Inactive,...,26.3,19.8,1.4,2.0,13.8,25.4,113,113,20.1,8.4
2022-12-09,NaN,25,18,37-344,LAL,@,PHI,L (-11),1,43:50,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
2022-12-11,NaN,26,19,37-346,LAL,@,DET,W (+7),1,36:52,...,6.1,21.9,2.2,0.0,10.5,28.7,92,122,13.2,-4.8


In [38]:
player_log.columns

Index(['Unnamed: 0', 'Rk', 'G', 'Age', 'Tm', 'Unnamed: 5', 'Opp', 'Unnamed: 7',
       'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc',
       '+/-', 'Unnamed: 0', 'Rk', 'G', 'Age', 'Tm', 'Unnamed: 5', 'Opp',
       'Unnamed: 7', 'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM'],
      dtype='object')

In [36]:
log_dict['Draymond Green']

{}

In [30]:
log_dict

{'Joel Embiid': {'last_season':             Unnamed: 0  Rk    G     Age   Tm Unnamed: 5  Opp Unnamed: 7 GS  \
  2021-10-20         0.0   1  1.0  27-218  PHI          @  NOP    W (+20)  1   
  2021-10-22         1.0   2  2.0  27-220  PHI        NaN  BRK     L (-5)  1   
  2021-10-24         2.0   3  3.0  27-222  PHI          @  OKC    W (+12)  1   
  2021-10-26         3.0   4  4.0  27-224  PHI          @  NYK    L (-13)  1   
  2021-10-28         4.0   5  5.0  27-226  PHI        NaN  DET     W (+8)  1   
  ...                ...  ..  ...     ...  ...        ...  ...        ... ..   
  2022-11-30         NaN  22   14  28-259  PHI          @  CLE    L (-28)  1   
  2022-12-02         NaN  23   15  28-261  PHI          @  MEM     L (-8)  1   
  2022-12-05         NaN  24   16  28-264  PHI          @  HOU     L (-9)  1   
  2022-12-09         NaN  25   17  28-268  PHI        NaN  LAL    W (+11)  1   
  2022-12-11         NaN  26   18  28-270  PHI        NaN  CHO    W (+18)  1   
  
       

In [17]:
player_log

,Unnamed: 0,Rk,G,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,BPM
2017-10-17,0.0,1,1.0,19-228,BOS,@,CLE,L (-3),1,36:32,...,13.7,13.4,0.0,0.0,7.0,17.3,115,108,11.7,-1.3
2017-10-18,1.0,2,2.0,19-229,BOS,NaN,MIL,L (-8),1,36:32,...,13.4,7.5,1.3,0.0,17.9,13.1,87,113,5.8,-6.7
2017-10-20,2.0,3,3.0,19-231,BOS,@,PHI,W (+10),1,32:45,...,11.5,0.0,0.0,7.3,20.1,18.7,113,90,11.7,4.3
2017-10-24,3.0,4,4.0,19-235,BOS,NaN,NYK,W (+21),1,31:08,...,7.3,13.3,6.5,4.2,16.7,28.1,112,85,18.9,11.7
2017-10-26,4.0,5,5.0,19-237,BOS,@,MIL,W (+7),1,27:49,...,4.2,18.6,1.8,0.0,19.6,17.1,125,95,10.3,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-05,NaN,25,24,24-277,BOS,@,TOR,W (+6),1,38:44,...,15.5,6.0,5.5,0.0,29.4,38.3,94,89,18.3,2.2
2022-12-07,NaN,26,25,24-279,BOS,@,PHO,W (+27),1,29:06,...,15.7,12.2,1.3,2.2,13.2,33.9,111,110,22.0,5.2
2022-12-10,NaN,27,26,24-282,BOS,@,GSW,L (-16),1,39:45,...,8.1,9.7,3.1,0.0,12.2,34.2,104,93,15.6,2.4
2022-12-12,NaN,28,27,24-284,BOS,@,LAC,L (-20),1,36:34,...,9.2,7.9,3.6,2.3,7.7,29.0,76,117,9.3,-6.2


In [10]:
player_log

,Unnamed: 0,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,GmSc,BPM
0,0.0,1,1.0,2017-10-17,19-228,BOS,@,CLE,L (-3),1,...,13.7,13.4,0.0,0.0,7.0,17.3,115,108,11.7,-1.3
1,1.0,2,2.0,2017-10-18,19-229,BOS,NaN,MIL,L (-8),1,...,13.4,7.5,1.3,0.0,17.9,13.1,87,113,5.8,-6.7
2,2.0,3,3.0,2017-10-20,19-231,BOS,@,PHI,W (+10),1,...,11.5,0.0,0.0,7.3,20.1,18.7,113,90,11.7,4.3
3,3.0,4,4.0,2017-10-24,19-235,BOS,NaN,NYK,W (+21),1,...,7.3,13.3,6.5,4.2,16.7,28.1,112,85,18.9,11.7
4,4.0,5,5.0,2017-10-26,19-237,BOS,@,MIL,W (+7),1,...,4.2,18.6,1.8,0.0,19.6,17.1,125,95,10.3,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,NaN,25,24,2022-12-05,24-277,BOS,@,TOR,W (+6),1,...,15.5,6.0,5.5,0.0,29.4,38.3,94,89,18.3,2.2
452,NaN,26,25,2022-12-07,24-279,BOS,@,PHO,W (+27),1,...,15.7,12.2,1.3,2.2,13.2,33.9,111,110,22.0,5.2
453,NaN,27,26,2022-12-10,24-282,BOS,@,GSW,L (-16),1,...,8.1,9.7,3.1,0.0,12.2,34.2,104,93,15.6,2.4
454,NaN,28,27,2022-12-12,24-284,BOS,@,LAC,L (-20),1,...,9.2,7.9,3.6,2.3,7.7,29.0,76,117,9.3,-6.2


In [18]:
pd.merge(main_lineup, player_season_stats, left_on = 'Name', right_on = 'Name')

,Name,ID,Position_x,Salary,TeamAbrev,Roster Position,StatID,TeamID,PlayerID,SeasonType,...,Steals,BlockedShots,Turnovers,PersonalFouls,Points,FantasyPointsFanDuel,FantasyPointsDraftKings,PlusMinus,DoubleDoubles,TripleDoubles
0,Luka Doncic,25809437,PG,12700,DAL,"[PG, G, UTIL]",1010874,25,20001984,1,...,42.0,16.0,89.0,71.0,827.0,1497.3,1561.50,97.0,14.0,6.0
1,Jayson Tatum,25809440,SF/PF,11100,BOS,"[SF, PF, F, UTIL]",1010118,9,20001829,1,...,30.0,26.0,74.0,61.0,801.0,1324.9,1381.00,214.0,13.0,0.0
2,Kevin Durant,25809444,SF/PF,11000,BKN,"[SF, PF, F, UTIL]",1011110,8,20000884,1,...,20.0,47.0,96.0,65.0,840.0,1400.1,1418.00,61.0,2.0,1.0
3,Ja Morant,25809448,PG,10600,MEM,"[PG, G, UTIL]",1011071,21,20002266,1,...,26.0,6.0,83.0,43.0,610.0,1055.0,1107.75,143.0,8.0,2.0
4,Shai Gilgeous-Alexander,25809457,PG,10100,OKC,"[PG, G, UTIL]",1010947,18,20001999,1,...,44.0,26.0,82.0,65.0,780.0,1259.9,1250.00,-5.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Daniel Theis,25810084,PF,3000,IND,"[PF, F, UTIL]",1010839,13,20001901,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
222,Dylan Windler,25810226,SG,3000,CLE,"[SG, G, UTIL]",1010855,12,20002296,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
223,Ricky Rubio,25810229,PG,3000,CLE,"[PG, G, UTIL]",1010955,12,20000797,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
224,Gary Payton II,25809830,PG/SG,3800,POR,"[PG, SG, G, UTIL]",1010952,19,20001730,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [35]:
fd.get_player_projection_stats_by_date(date = '2022-DEC-12')

https://api.sportsdata.io/api/nba/fantasy/json/PlayerGameProjectionStatsByDate/2022-DEC-12


,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,Started,InjuryStatus,...,Steals,BlockedShots,Turnovers,PersonalFouls,Points,FantasyPointsFanDuel,FantasyPointsDraftKings,PlusMinus,DoubleDoubles,TripleDoubles
0,1034229,4,20000457,1,2023,Kyle Lowry,MIA,PG,1,None,...,1.24,0.38,2.26,2.83,12.70,29.45,30.31,0.0,0.0,0.0
1,1034230,4,20000515,1,2023,Jimmy Butler,MIA,SF,1,Probable,...,1.36,0.44,1.75,1.53,22.14,40.20,39.99,0.0,0.0,0.0
2,1034231,4,20000684,1,2023,Dewayne Dedmon,MIA,C,0,Probable,...,0.18,0.47,0.62,2.17,4.63,10.43,10.37,0.0,0.0,0.0
3,1034232,4,20000689,1,2023,Victor Oladipo,MIA,SG,0,Probable,...,0.91,0.19,1.54,2.09,11.17,21.08,21.60,0.0,0.0,0.0
4,1034233,4,20000780,1,2023,Udonis Haslem,MIA,PF,0,None,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,1034457,28,20002352,1,2023,Moses Brown,LAC,C,0,None,...,0.11,0.33,0.32,1.03,3.24,7.41,7.26,0.0,0.0,0.0
229,1034458,28,20002386,1,2023,Amir Coffey,LAC,PF,0,None,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
230,1034459,28,20002689,1,2023,Jason Preston,LAC,PG,0,None,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
231,1034460,28,20002690,1,2023,Brandon Boston Jr.,LAC,SG,0,None,...,0.36,0.21,0.52,0.72,5.92,10.68,10.77,0.0,0.0,0.0
